In [1]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import json

In [2]:
# Load the pre-trained emotion analysis pipeline
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

Device set to use mps:0


In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
print("Model Token Limit:", tokenizer.model_max_length)

Model Token Limit: 512


In [3]:
with open("final_campaign_data.txt", "r") as file:
    data = json.load(file)

In [53]:
# Convert the JSON data to a Pandas DataFrame
campaigns = pd.DataFrame(data)

In [52]:
# for index, row in campaigns.iterrows():
#     description = " ".join(row["Description"].split())
#     tokens = tokenizer(description, return_tensors="pt", truncation=False)["input_ids"][0]
#     chunks = [tokens[i: i + 20] for i in range(0, len(tokens), 10)]
#     print(chunks[0])
#     print(chunks[1])
#     break

tensor([    0, 43309,  3869,  2981, 21721,   571, 10564,    16, 13916,    42,
        11137,     4,  2497,   183,    82,     9,   476,    25,     5,  1270])
tensor([11137,     4,  2497,   183,    82,     9,   476,    25,     5,  1270,
          161,   112,  1404,     7,    65,   153,  1932,    38,   524,  5162])


In [13]:

# def classify_emotion(description):
#     try:
#         result = emotion_pipeline(description, emotionally_charged_threshold)
#         dominant_emotion = max(result, key=lambda x: x['score'])
#         is_emotionally_charged = (
#             dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral"
#         )
#         return dominant_emotion['label'], dominant_emotion['score'], is_emotionally_charged
#     except Exception as e:
#         return "Error", 0.0, False

TOKEN_LIMIT = tokenizer.model_max_length

# Function to classify emotions with a sliding window approach for long texts
def classify_emotion_sliding_window(description, emotionally_charged_threshold):
    try:
        description = " ".join(description.split())
        # Tokenize the text to check length
        tokens = tokenizer(description, return_tensors="pt", truncation=False)["input_ids"][0]

        if len(tokens) <= TOKEN_LIMIT:
            # If text fits within the token limit, classify normally
            result = emotion_pipeline(description)
            dominant_emotion = max(result, key=lambda x: x['score'])
            is_emotionally_charged = dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral"
            return dominant_emotion['label'], dominant_emotion['score'], is_emotionally_charged

        # If text exceeds the token limit, apply sliding window approach
        stride = TOKEN_LIMIT // 2  # Overlapping stride of 50%
        chunks = [tokens[i: i + TOKEN_LIMIT] for i in range(0, len(tokens), stride)]

        highest_emotion = None
        highest_score = 0.0
        is_emotionally_charged = False

        for chunk in chunks:
            decoded_text = tokenizer.decode(chunk, skip_special_tokens=True)
            result = emotion_pipeline(decoded_text)
            dominant_emotion = max(result, key=lambda x: x['score'])

            # Check if this chunk has the highest emotion score so far
            if dominant_emotion['score'] > highest_score:
                highest_emotion = dominant_emotion['label']
                highest_score = dominant_emotion['score']
                print(decoded_text, highest_emotion, highest_score)

            # If any chunk has a score higher than the threshold, mark as emotionally charged
            if dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral":
                is_emotionally_charged = True

        return highest_emotion, highest_score, is_emotionally_charged

    except Exception as e:
        return "Error", 0.0, False


In [17]:
len(campaigns)

21993

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import time

emotionally_charged_threshold = 0.70
start_time = time.time()

# Perform emotion analysis on all campaign title
campaigns1 = campaigns.head(20)
campaigns1[["DominantEmotion", "EmotionScore", "EmotionallyCharged"]] = campaigns1["Description"].apply(
    lambda desc: pd.Series(classify_emotion(desc, emotionally_charged_threshold))
)
end_time = time.time()
execution_time = end_time - start_time  # Compute elapsed time
print(f"Execution Time: {execution_time:.6f} seconds")

Execution Time: 0.522785 seconds


In [16]:
campaigns1

,_id,URL,Category,image,GoalAmount,Organizer,Description,Updates,Comments,Created,Data_created_on,success,Title,RaisedAmount,DominantEmotion,EmotionScore,EmotionallyCharged
21973,{'$oid': '61e44499697f4b953f55bdb8'},https://www.gofundme.com/f/zurqk6-hayden-baxley,"Babies, Kids & Family","background-image: url(""https://images.gofundme...",5000,"{'Organizer_name': 'Samantha Fralish', 'Organi...",Samantha Fralish is organizing this fundraiser...,[],"[{'donated_amount': 100, 'comment': 'Very few ...","Created November 1, 2021",{'$date': {'$numberLong': '1642331721769'}},1,Hayden Baxley,7210.0,sadness,0.976758,True
21974,{'$oid': '61ea22533d2cac2f33b66782'},https://www.gofundme.com/f/zuv43-if-i-could-talk,Animals & Pets,"background-image: url(""https://images.gofundme...",5000,"{'Organizer_name': 'Theresa Chapman', 'Organiz...",Theresa Chapman is organizing this fundraiser....,[],[],"Created November 8, 2021",{'$date': {'$numberLong': '1642716163190'}},0,If I could talk...,1890.0,Error,0.000000,False
21975,{'$oid': '61e6028a1ad7d6fcca9ad8cd'},https://www.gofundme.com/f/zv7ng-harvard-chris...,Community & Neighbors,"background-image: url(""https://images.gofundme...",20000,"{'Organizer_name': 'Heather Tody-Schamper', 'O...",Team fundraiser\nHeather Tody-Schamper and 0 o...,"[November 30, 2021, by Joe McLachlan, Benefici...",[],"Created September 20, 2021",{'$date': {'$numberLong': '1642445882494'}},0,Harvard Christmas Giving Tree,925.0,joy,0.572194,False
21976,{'$oid': '61e21511697f4b953f55b1d7'},https://www.gofundme.com/f/zvgzj-christmas-mir...,"Missions, Faith & Church","background-image: url(""https://images.gofundme...",1000,"{'Organizer_name': 'Obey Keya', 'Organizer_des...",Obey Keya is organizing this fundraiser.\n\n\n...,[],[],"Created September 11, 2021",{'$date': {'$numberLong': '1642188481456'}},0,Christmas Miracle,450.0,neutral,0.355818,False
21977,{'$oid': '61e1146b1ad7d6fcca9abced'},https://www.gofundme.com/f/zvmz7-last-hope-to-...,"Medical, Illness & Healing","background-image: url(""https://images.gofundme...",200000,"{'Organizer_name': 'Agnieszka Wiak', 'Organize...",Agnieszka Wiak is organizing this fundraiser.\...,[],"[{'donated_amount': 100, 'comment': 'Pani Patr...","Created November 15, 2021",{'$date': {'$numberLong': '1642122779500'}},0,Last hope to save Patrycja's life,7790.0,Error,0.000000,False
21978,{'$oid': '61e507261ad7d6fcca9ad3f2'},https://www.gofundme.com/f/zvrdtj-olson-family,Community & Neighbors,"background-image: url(""https://images.gofundme...",10000,"{'Organizer_name': 'Amanda Nichols', 'Organize...",Team fundraiser\nAmanda Nichols and Kara Breue...,[],"[{'donated_amount': 25, 'comment': 'In my Aunt...","Created September 25, 2021",{'$date': {'$numberLong': '1642381526348'}},1,Olson Family Fundraiser,10497.0,sadness,0.459890,False
21979,{'$oid': '61e347aa3d2cac2f33b64793'},https://www.gofundme.com/f/zw88d-help-our-dad-...,Accidents & Emergencies,"background-image: url(""https://images.gofundme...",20000,"{'Organizer_name': 'Miriam Castillo', 'Organiz...",Miriam Castillo is organizing this fundraiser....,"[December 13, 2021, by Miriam Castillo, Organi...","[{'donated_amount': 200, 'comment': 'Que Dios ...","Created November 7, 2021",{'$date': {'$numberLong': '1642266970121'}},0,Help our dad fight cancer,13733.0,Error,0.000000,False
21980,{'$oid': '61e90ae23d2cac2f33b66240'},https://www.gofundme.com/f/zwck5q-help-us-save...,Animals & Pets,"background-image: url(""https://images.gofundme...",5000,"{'Organizer_name': 'Donna Stomber', 'Organizer...",Donna Stomber is organizing this fundraiser.\n...,[],"[{'donated_amount': 25, 'comment': 'Speedy rec...","Created September 15, 2021",{'$date': {'$numberLong': '1642644626054'}},0,Help Us Save Our Puppy,2737.0,neutral,0.546312,False
21981,{'$oid': '61e213353d2cac2f33b6426a'},https://www.gofundme.com/f/zwuywq-help-brandon...,Accidents & Emergencies,"background-image: url(""https://images.gofundme...",5000,"{'Organizer_name': 'Alyssa Darbyshire', 'Organ...